# 第一个cell为测试，请运行第二、三个cell处理

In [39]:
import pandas as pd
import re

# Adjusted code snippet based on the uploaded Excel file's new name
def extract_hour(hour_str):
    match = re.match(r'(\d+)', hour_str)
    if match:
        return int(match.group(1))
    return None

# Load the Excel file
new_xls_corrected = pd.ExcelFile("/Users/zhiranbai/Downloads/COPY OF BA系统设备整点报表202308.xlsx")

# Initialize a list to hold the processed data
ordered_data = []

# Process each sheet
for sheet_name in new_xls_corrected.sheet_names:
    # Extract month and day from the sheet name
    month_day_match = re.match(r'(\d+)月(\d+)日', sheet_name)
    if month_day_match:
        month, day = map(int, month_day_match.groups())
        df = pd.read_excel(new_xls_corrected, sheet_name=sheet_name, header=None)
        
        # Extract hour information and parameter names
        hours = [extract_hour(str(hour)) for hour in df.iloc[0, 2:].values if extract_hour(str(hour)) is not None]
        parameters = df.iloc[1:, 1].values  # Parameters start from row 1, column B
        
        # Aggregate data
        for hour_col, hour in enumerate(hours):
            for param_row, parameter in enumerate(parameters):
                # Ensure we're not exceeding the bounds of the DataFrame
                if param_row < df.shape[0] and hour_col + 2 < df.shape[1]:
                    value = df.iloc[param_row + 1, hour_col + 2]  # Values start from column C
                    
                    date_time_str = f"{month:02d}-{day:02d} {hour:02d}"
                    ordered_data.append([date_time_str, parameter, value])

final_ordered_df = pd.DataFrame(ordered_data, columns=['MM-DD hour', 'Parameter', 'Value'])
final_ordered_df
# final_ordered_output_path = '/Users/zhiranbai/Downloads/final_ordered_combined_data1.xlsx'
# final_ordered_df.to_excel(final_ordered_output_path, index=False)
# pivoted_df = final_ordered_df.pivot(index='MM-DD hour', columns='Parameter', values='Value')
# pivot_df.reset_index(inplace=True)
# pivoted_df
# # Initialize a dictionary to store unique 'MM-DD hour' values along with parameter-value pairs
# unique_hours_data = {}

# # Populate the dictionary with unique 'MM-DD hour' values and parameters
# for date_hour, parameter, value in ordered_data:
#     if date_hour not in unique_hours_data:
#         unique_hours_data[date_hour] = {}
#     unique_hours_data[date_hour][str(parameter)] = value

# # Prepare the structure for the final DataFrame
# all_parameters_set = {str(param) for daily_data in unique_hours_data.values() for param in daily_data.keys()}

# # Create rows for the DataFrame
# rows_list_corrected = []
# for date_hour, params_values in unique_hours_data.items():
#     row = [date_hour] + [params_values.get(param, None) for param in all_parameters_set]
#     rows_list_corrected.append(row)

# # Create the final DataFrame with 'MM-DD hour' and parameters spread horizontally
# final_transformed_df_corrected = pd.DataFrame(rows_list_corrected, columns=["MM-DD hour"] + sorted(all_parameters_set))

# final_transformed_df_corrected


,MM-DD hour,Parameter,Value
0,08-14 00,室外湿度（%）,NaN
1,08-14 00,室外湿球（℃）,26.9
2,08-14 00,室外干球（℃）,26.9
3,08-14 00,运行模式,冷机模式
4,08-14 00,冷机冷冻回水温度（℃）,17.7
...,...,...,...
383563,04-02 23,集水器1回水温度（℃）,NaN
383564,04-02 23,集水器2回水温度（℃）,NaN
383565,04-02 23,分水器1供水温度（℃）,NaN
383566,04-02 23,分水器2供水温度（℃）,NaN


## 请修改 源excel ‘new_xls_corrected’ 和 输出文件位置 ‘final_ordered_output_path’
## 源excel 需要另存为一下（变成可修改），要不会报错

In [6]:
import pandas as pd
import numpy as np
import re

def extract_hour(hour_str):
    """Extracts the numeric hour part from a string."""
    match = re.match(r'(\d+)', hour_str)
    if match:
        return int(match.group(1))
    return None

def extract_number(value):
    """Extracts the numeric part of a string, including sequences separated by slashes, excluding any trailing letters."""
    # Matches a number optionally followed by more numbers separated by slashes, 
    # capturing the entire sequence while excluding any letters that may follow.
    match = re.match(r"(\d*\.?\d+(?:/\d*\.?\d+)*)", str(value))
    return match.group(1) if match else None

ordered_data = []
new_xls_corrected = pd.ExcelFile("/Users/zhiranbai/Downloads/BA系统设备整点报表202308.xlsx")
for sheet_name in new_xls_corrected.sheet_names:
    month_day_match = re.match(r'(\d+)月(\d+)日', sheet_name)
    if month_day_match:
        month, day = map(int, month_day_match.groups())
        df = pd.read_excel(new_xls_corrected, sheet_name=sheet_name, header=None)
        
        hours = [extract_hour(str(hour)) for hour in df.iloc[0, 2:].values if extract_hour(str(hour)) is not None]
        
        current_system_identifier = None  # Initialize with no system identifier
        
      
            
            
        for hour_col, hour in enumerate(hours):
            for row_index in range(1, df.shape[0]):
                system_identifier_cell = df.iloc[row_index, 0]
                if pd.notnull(system_identifier_cell) and str(system_identifier_cell).strip():  # Check for a new system identifier
                    current_system_identifier = str(system_identifier_cell).strip()  # Update the system identifier
                if row_index == 4:
                    current_system_identifier='1#系统'
                parameter = df.iloc[row_index, 1]  # Parameter names are in the second column (index 1)
                if hour_col + 2 < df.shape[1]:
                    value = df.iloc[row_index, hour_col + 2]
                    value = extract_number(value)
                    if row_index == 4:
                        value =value
                        
                    date_time_str = f"{month:02d}-{day:02d} {hour:02d}"
                    ordered_data.append([date_time_str, current_system_identifier, parameter, value])

final_ordered_df = pd.DataFrame(ordered_data, columns=['MM-DD hour', 'System Identifier', 'Parameter', 'Value'])
parameter_mapping = {
    '冷机冷冻回水温度（℃）': '冷机冷冻回水温度',
    '冷机冷冻出水温度（℃）': '冷机冷冻出水温度',
    '板换冷冻回水温度（℃）': '板换冷冻回水温度',
    '板换冷冻出水温度（℃）': '板换冷冻出水温度',
    '冷机冷却回水温度（℃）': '冷机冷却回水温度',
    '冷机冷却出水温度（℃）': '冷机冷却出水温度',
    '板换冷却回水温度（℃）': '板换冷却回水温度',
    '板换冷却出水温度（℃）': '板换冷却出水温度',
    '冷塔出水温度（℃）': '冷塔出水温度',
    '冷塔回水温度（℃）': '冷塔回水温度',
    '冷机负载率（%）': '冷机负载率',
    '冷机功率（KW）': '冷机功率',
    '冷冻水频率（HZ）': '冷冻水频率',
    '冷冻水功率':'冷冻水泵功率（KW）',
    '冷冻水功率（KW）':'冷冻水泵功率（KW）',
    '冷冻水频率':'冷冻水泵频率（HZ）',
    
    '冷却水频率（HZ）': '冷却水频率',
    '冷却水频率':'冷却水泵频率（HZ）',
    '冷却水功率（KW）': '冷却水功率',
    '冷塔频率（HZ）': '冷塔频率',
    '冷塔功率（KW）': '冷塔功率',
    '冷却水泵功率（KW）':'冷却水功率'
    
}

# Apply the mapping to standardize parameter names
final_ordered_df['Parameter'] = final_ordered_df['Parameter'].map(parameter_mapping).fillna(final_ordered_df['Parameter'])

final_ordered_df['group'] = ((final_ordered_df['MM-DD hour'] != final_ordered_df['MM-DD hour'].shift()) | (final_ordered_df['System Identifier'] != final_ordered_df['System Identifier'].shift())).cumsum()

df_indexed = final_ordered_df.pivot_table(index=['group', 'MM-DD hour', 'System Identifier'], columns='Parameter', values='Value', aggfunc='first').reset_index()

# Removing 'group' from the index as it's only a temporary column used for grouping
df_indexed.drop(columns='group', inplace=True)

# Set 'System Identifier' and 'MM-DD hour' as a multi-level index
df_indexed.set_index(['MM-DD hour', 'System Identifier'], inplace=True)


outdoor_params = final_ordered_df[final_ordered_df['System Identifier'] == '室外状态'].drop(columns='group')
outdoor_pivot = outdoor_params.pivot_table(index='MM-DD hour', columns='Parameter', values='Value', aggfunc='first')

# Find overlapping columns and rename them in outdoor_pivot to prevent suffix appending
overlap_cols = set(df_indexed.columns).intersection(set(outdoor_pivot.columns))
for col in overlap_cols:
    outdoor_pivot.rename(columns={col: f"{col}_室外状态"}, inplace=True)

# Now join the '室外状态' parameters with the main DataFrame without redundant suffixes
df_combined = df_indexed.join(outdoor_pivot, on='MM-DD hour')



df_combined=df_combined.drop(columns='室外干球（℃）')
df_combined=df_combined.drop(columns='室外湿度（%）')
df_combined=df_combined.drop(columns='室外湿球（℃）')
df_combined = df_combined.drop('室外状态', level='System Identifier')
df_combined['冷冻水泵频率（HZ）'] = df_combined['冷冻水泵频率（HZ）'].combine_first(df_combined['冷冻水频率'])
df_combined['冷却水泵频率（HZ）'] = df_combined['冷却水泵频率（HZ）'].combine_first(df_combined['冷却水频率'])
df_combined=df_combined.drop(columns='冷冻水频率')
df_combined=df_combined.drop(columns='冷却水频率')

ordered_columns = [
    "运行模式", "冷机冷冻回水温度", "冷机冷冻出水温度", "板换冷冻回水温度", "板换冷冻出水温度",
    "冷机冷却回水温度", "冷机冷却出水温度", "板换冷却回水温度", "板换冷却出水温度", "冷塔出水温度",
    "冷塔回水温度", "冷机负载率", "冷机功率", "冷冻水泵频率（HZ）", "冷冻水泵功率（KW）",
    "冷却水泵频率（HZ）", "冷却水功率", "冷塔频率", "冷塔功率", "冷冻水流量（m³/h)",
    "冷却水流量（m³/h)", "冷机电流百分比（%）", "集水器1回水温度（℃）", "集水器2回水温度（℃）",
    "分水器1供水温度（℃）", "分水器2供水温度（℃）", "压差（KPA）", "室外干球（℃）_室外状态",
    "室外湿度（%）_室外状态", "室外湿球（℃）_室外状态"
]

# Reorder df_combined to have columns in the specified order
# Use .reindex() method with axis=1 for columns, and align to your ordered_columns list
df_combined = df_combined.reindex(ordered_columns, axis=1)

rename_mapping = {
    "室外干球（℃）_室外状态":'室外干球',
    "室外湿度（%）_室外状态":'室外湿度',
    "室外湿球（℃）_室外状态":'室外湿球',
    '冷冻水泵频率（HZ）':'冷冻水泵频率',
    '冷冻水泵功率（KW）':'冷冻水泵功率',
    '冷却水泵频率（HZ）':'冷却水泵频率',
    '冷冻水流量（m³/h)':'冷冻水流量',
    '冷却水流量（m³/h)':'冷却水流量',
    '冷机电流百分比（%）':'冷机电流百分比',
    '集水器1回水温度（℃）':'集水器1回水温度',
    '集水器2回水温度（℃）':'集水器2回水温度',
    '分水器1供水温度（℃）':'分水器1供水温度',
    '分水器2供水温度（℃）':'分水器2供水温度',
    '压差（KPA）':'压差',
    '冷却水功率':'冷却水泵功率'
    
}

# Apply the renaming
df_combined = df_combined.rename(columns=rename_mapping)
final_ordered_output_path = '/Users/zhiranbai/Downloads/整点报表处理数据.xlsx'
# to save to Excel:
df_combined.to_excel(final_ordered_output_path, index=True)

# Load the Excel file
file_path = final_ordered_output_path  # Update this to your Excel file's path
df = pd.read_excel(file_path)

# Initialize a variable to keep track of the current 'MM-DD hour' value
current_mm_dd_hour = None

# Go through each row in Column A
for index, row in df.iterrows():
    if pd.notnull(row['MM-DD hour']):  # Assuming 'A' is the column name for 'MM-DD hour'
        current_mm_dd_hour = row['MM-DD hour']  # Update current 'MM-DD hour' if the cell is not null
    else:
        df.at[index, 'MM-DD hour'] = current_mm_dd_hour  # Fill the null cell with the current 'MM-DD hour'




# Save the modified DataFrame back to an Excel file
output_path = file_path  # Update this to your desired output path
df.to_excel(output_path, index=False)


In [8]:
from datetime import timedelta
final_ordered_output_path = '/Users/zhiranbai/Downloads/整点报表处理数据.xlsx'


# Load the Excel file
file_path = final_ordered_output_path  # Update this to your Excel file's path
df = pd.read_excel(file_path)

# Initialize a variable to keep track of the current 'MM-DD hour' value
current_mm_dd_hour = None

# Go through each row in Column A
for index, row in df.iterrows():
    if pd.notnull(row['MM-DD hour']):  # Assuming 'A' is the column name for 'MM-DD hour'
        current_mm_dd_hour = row['MM-DD hour']  # Update current 'MM-DD hour' if the cell is not null
    else:
        df.at[index, 'MM-DD hour'] = current_mm_dd_hour  # Fill the null cell with the current 'MM-DD hour'


df.dropna(subset=[
    "运行模式", "冷机冷冻回水温度", "冷机冷冻出水温度", "板换冷冻回水温度", "板换冷冻出水温度",
    "冷机冷却回水温度", "冷机冷却出水温度", "板换冷却回水温度", "板换冷却出水温度", "冷塔出水温度",
    "冷塔回水温度", "冷机负载率", "冷机功率", "冷冻水泵频率", "冷冻水泵功率", "冷却水泵频率",
    "冷却水泵功率", "冷塔频率", "冷塔功率", "冷冻水流量", "冷却水流量", "冷机电流百分比",
    "集水器1回水温度", "集水器2回水温度", "分水器1供水温度", "分水器2供水温度", "压差"
], how='all', inplace=True)

# 定义检查非空值的函数
def check_not_null(row, columns):
    return not row[columns].isnull().all()

df['运行模式'] = df['运行模式'].astype('object') 

# 遍历 DataFrame 的每一行来更新运行模式
for index, row in df.iterrows():
    # 冷机模式检查条件
    chiller_condition_columns = ["冷机冷冻回水温度", "冷机冷冻出水温度", "冷机冷却回水温度", "冷机冷却出水温度", "冷机负载率", "冷机功率"]
    # 板换模式检查条件
    plate_exchange_condition_columns = ["板换冷却回水温度", "板换冷却出水温度", "板换冷冻回水温度", "板换冷冻出水温度"]
    
    # 检查是否符合混合模式
    if check_not_null(row, chiller_condition_columns) and check_not_null(row, plate_exchange_condition_columns):
        df.at[index, '运行模式'] = '混合模式'
        if row['冷机功率']==0:
            df.at[index, '运行模式'] = '板换模式'
    # 检查是否符合冷机模式
    elif check_not_null(row, chiller_condition_columns):
        df.at[index, '运行模式'] = '冷机模式'
    # 检查是否符合板换模式
    elif check_not_null(row, plate_exchange_condition_columns):
        df.at[index, '运行模式'] = '板换模式'
        


# Initial year
year = 2023

# Function to adjust the year based on month transitions
def adjust_year_and_hour(row, current_year):
    global year
    # Extract month and hour from the row
    parts = row['MM-DD hour'].split(' ')
    month = int(parts[0].split('-')[0])
    hour = parts[1]
    
    # If it's January and the previous month was December, increment the year
    if month == 1 and df.iloc[row.name - 1]['month'] == 12:
        year += 1
    
    # Adjust for '24' hour notation
    date_str = f"{year}-{row['MM-DD hour']}"
    if hour == '24':
        # Correct the hour and add a day to the date
        corrected_date = pd.to_datetime(date_str.replace(' 24', ' 00'), format="%Y-%m-%d %H") + timedelta(days=1)
    else:
        corrected_date = pd.to_datetime(date_str, format="%Y-%m-%d %H")
        
    return corrected_date

# Add a temporary column to assist with detecting month changes
df['month'] = df['MM-DD hour'].apply(lambda x: int(x.split('-')[0]))

# Apply the adjust_year_and_hour function to each row to handle year transition and correct hour
df['MM-DD hour'] = df.apply(adjust_year_and_hour, axis=1, current_year=year)

# Drop the temporary 'month' column as it's no longer needed
df.drop('month', axis=1, inplace=True)



        
# Save the modified DataFrame back to an Excel file
output_path = '/Users/zhiranbai/Downloads/整点报表处理数据_new.xlsx'  # Update this to your desired output path
df.to_excel(output_path, index=False)